In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

In [2]:
data = pd.read_csv('iris_data.csv')# Load the CSV data
X = data[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']].values# features, this are the input variables
y = data[['SetosaScore', 'VersicolorScore', 'VirginicaScore']].values# targets 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)# split the data into train (80%) and test (20%) sets

scaler = StandardScaler()#standardize the features 
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype = torch.float32) #convert data to PyTorch tensors
X_test_tensor = torch.tensor(X_test, dtype = torch.float32) 
y_train_tensor = torch.tensor(y_train, dtype = torch.float32)
y_test_tensor = torch.tensor(y_test, dtype = torch.float32)

batch_size = 8 # Create PyTorch DataLoader objects for batching
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle = False)

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 512)  # 512 hidden units
        self.fc6 = nn.Linear(512, 3)  # Output layer for 3 target scores

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc6(x)
        return x
# Initialize the model
model = NeuralNetwork()

In [4]:
#define the loss function and optimizer
criterion = nn.L1Loss()# MAE Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [5]:
num_epochs = 10#train the model
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    epoch_loss = 0

    for inputs, targets in train_loader:
        optimizer.zero_grad()  # reset gradients
        outputs = model(inputs)  # forward pass
        loss = criterion(outputs, targets)  #compute loss
        loss.backward()  # backpropagation
        optimizer.step()  #update weights

        epoch_loss += loss.item()
    avg_loss = epoch_loss / len(train_loader)#compute the average loss for the epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

Epoch [1/10], Loss: 0.4090
Epoch [2/10], Loss: 0.3278
Epoch [3/10], Loss: 0.2994
Epoch [4/10], Loss: 0.2881
Epoch [5/10], Loss: 0.2769
Epoch [6/10], Loss: 0.2920
Epoch [7/10], Loss: 0.2930
Epoch [8/10], Loss: 0.2874
Epoch [9/10], Loss: 0.2959
Epoch [10/10], Loss: 0.2862


In [6]:
model.eval()  # Set model to evaluation mode
test_loss = 0

with torch.no_grad():  # Disable gradient computation for evaluation
    for inputs, targets in test_loader:# Iterate over the test set
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()

avg_test_loss = test_loss / len(test_loader)
print(f'Test Loss: {avg_test_loss:.4f}')

Test Loss: 0.2051


In [7]:

sample1 = torch.tensor([[7, 4, 7, 3]], dtype=torch.float32)# Inputs 
with torch.no_grad():# Do prediction
    pred1 = model(sample1)
    print(f'Sample1 Predictions: {pred1.numpy()}')


Sample1 Predictions: [[-2.4416459  0.2791353  2.2075915]]
